In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [27]:
from langchain_groq import ChatGroq

chat = ChatGroq(model = "mixtral-8x7b-32768", api_key=groq_api_key)

In [4]:
from langchain_community.tools import TavilySearchResults

# Configure Tavily tool to search only in Indian legal databases
tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
    include_domains=[
        "indiankanoon.org",        # Indian case law
        "barandbench.com",         # Legal news and updates
        "legallyindia.com",        # Legal developments in India
        "scconline.com",           # Supreme Court Cases Online
        "lawtimesjournal.in",      # Legal news and case analysis
        "lawyersclubindia.com",    # Legal community discussions
        "vlex.in",                 # Global legal information with Indian focus
        "taxmann.com"              # Taxation and corporate law in India
    ],
    # Optionally, exclude other irrelevant sources
    exclude_domains=["globalsearch.com", "genericnews.com"]
)


In [14]:
# Function to extract text from PDFs in a directory
from PyPDF2 import PdfReader
def extract_text_from_pdfs(directory_path):
    extracted_text = ""
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                for page in pdf_reader.pages:
                    extracted_text += page.extract_text() + "\n\n"
    return extracted_text

In [5]:
#RAG method
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Function to load and chunk PDF documents
def load_and_chunk_pdfs(directory_path):
    docs = []
    
    # Iterate through all PDFs in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            
            # Load the PDF
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            
            # Create a Document object from the extracted text
            doc = Document(page_content=text, metadata={"source": filename})
            docs.append(doc)
    
    # Set up the text splitter to chunk documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    
    # Split the loaded documents into chunks
    chunked_docs = text_splitter.split_documents(docs)
    return chunked_docs


In [6]:
# Extract text from PDFs
pdf_directory = "input"  # Replace with your actual directory path
pdf_content = load_and_chunk_pdfs(pdf_directory)

In [7]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_token = os.getenv("HF_TOKEN")

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=hf_token, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

d:\LawNavigator\venv\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
#Setting up the vector database
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore(embedding=embeddings)
vectorstore.add_documents(pdf_content)

['b5959372-f029-4464-8431-6bcdc71c57a9',
 'a50c3962-b0e9-4b2c-a736-03155eb02c8f',
 'ee9b33ea-40c6-46da-b53c-e17984c61883',
 '120d0c34-2a8e-44ce-b7ff-e9b091fc8636']

In [9]:
retriever = vectorstore.as_retriever()

In [10]:
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(
    retriever,
    "Pdf_content_retriever",
    "Searches and returns excerpts from the set of pdf docs.",
)

In [16]:
advisor_template = """You are a legal research assistant tasked with providing 
legal advice based on the given vectorstore context. If needed, conduct 
additional research using the Tavily Search tool. Analyze the query for 
specific legal issues, reference relevant sections of legal documents, and 
ensure jurisdictional relevance. Consider conflicting interpretations or 
unclear areas of law, and provide practical recommendations or next steps. 
Include a disclaimer regarding the limitations of AI-generated legal advice."""


In [12]:
tools = [tavily_tool, retrieval_tool]

In [28]:
# Define the graph

from langgraph.prebuilt import create_react_agent

advisor_graph = create_react_agent(chat, tools=tools, state_modifier=advisor_template)

In [29]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [30]:
inputs = {"messages": [("user", "What are the legal options for Fast Retail Pvt. Ltd. in this breach of contract case?")]}

async for chunk in advisor_graph.astream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What are the legal options for Fast Retail Pvt. Ltd. in this breach of contract case?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_caqj)
 Call ID: call_caqj
  Args:
    query: legal options for Fast Retail Pvt. Ltd. in breach of contract case
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://indiankanoon.org/doc/111578719/", "content": "Thus, since the defendant has failed to discharge the aforesaid onus, it can be safely concluded that the plaintiff vide email dated 28.02.2020 Ex.PW1/2 had offered the possession of the suit property and since the lease was to commence from 01.03.2020 and admittedly rent for the month of March also stood paid, the constructive possession of the suit property had vested with the defendant. It is pellucid from 

In [21]:
predictor_template = """
You are a legal research assistant. Your task is to predict the outcome of a legal case based on the given context and by conducting additional research using the Tavily Search tool when necessary.
Context from the vectorstore
Use this information as your primary source. If you need more recent or specific information, use the Tavily Search tool.
Remember to:
1. Analyze the case for relevant legal precedents, evidence, and arguments.
2. Reference sections from the provided legal documents that support your prediction.
3. Use the Tavily search tool for up-to-date case law or relevant outcomes.
4. Provide your prediction of the case outcome in terms of confidence intervals (e.g., 70% chance of a favorable outcome for the plaintiff).
5. Consider any jurisdictional differences and mention which jurisdiction your prediction pertains to.
6. Highlight any factors that might create uncertainty or lead to different outcomes.
7. Include a disclaimer about the limitations of AI-generated predictions and the factors that might influence real-world decisions.
"""


In [22]:
predictor_graph = create_react_agent(chat, tools=tools, state_modifier=predictor_template)

In [23]:
inputs = {"messages": [("user", "
Predict the likely outcome of Fast Retail's ₹80,00,000 breach of contract lawsuit against Tech Solutions.")]}
print_stream(predictor_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Predict the likely outcome of Fast Retail's ₹80,00,000 breach of contract lawsuit against Tech Solutions.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_gg7j)
 Call ID: call_gg7j
  Args:
    query: Fast Retail ₹80,00,000 breach of contract lawsuit against Tech Solutions recent court decisions
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.taxmann.com/post/blog/basic-primer-on-the-indian-contract-act", "content": "Mar 24, 2023 · A comprehensive guide on the Indian Contract Act, which is the law governing contracts in India, covers the key definitions and principles ...Missing:  Fast lawsuit Solutions decisions"}]
================================== Ai Message ==================================

Based on the information provided and the res